# Assignment2 - Supervised Learning flow

# Part 1(a) Student details:
* Please write the First-Name, First letter of Last-Name and last 4 digits of the i.d. for each student. 

In [8]:
# stundent details example: John S. 9812
#                       student details 1: Ilay W. 0725
# (if exists)           student details 2: Aharoni C. 1047
# (if exists)           student details 3: Adir B.
# (if exists&premitted) student details 4: 

## Part 1(b) - Chat-GPT/other AI-agent/other assistance used:
* If you changed the prompt until you got a satisfying answer, please add all versions
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add information:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>   
AI agent name:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 


AI agent name 2:
Goal:
Propmpt1:
    
Propmpt2:
    
Propmpt3: 

Other assistanse:    
</pre>

## Part 1(c) - Learning Problem and dataset explaination.
* Please explain in one paragraph
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add explaining text:

#### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>




    
</pre>

## Part 2 - Initial Preparations 
You could add as many code cells as needed

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer

In [10]:
train_df = pd.read_csv('titanic_train.csv')
test_df = pd.read_csv('titanic_test.csv')

print('First 5 training rows:')
print(train_df.head())
print('\nFirst 5 test rows:')
print(test_df.head())
print('\n Aharoni is gay')

First 5 training rows:
   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  Survived
0       1    0  54.0      1      0  78.2667         1         1
1       2    0  45.0      0      0  13.5000         0         1
2       3    1  20.0      0      0   9.5000         0         0
3       3    1  36.0      0      0   7.8958         0         0
4       3    1  22.0      0      0   8.0500         0         0

First 5 test rows:
   Pclass  Sex        Age  SibSp  Parch     Fare  Embarked  Survived
0       3    1  35.000000      0      0   7.0500         0         0
1       3    1  30.000000      0      0   7.2500         0         0
2       3    1  29.699118      0      0  14.5000         0         0
3       3    0  21.000000      0      0   7.7500         2         0
4       3    1  32.000000      0      0  56.4958         0         1

 Aharoni is gay


In [11]:
sns.set(style='whitegrid')

# Survival counts by passenger class
plt.figure(figsize=(6, 4))
sns.countplot(data=train_df, x='Pclass', hue='Survived')
plt.title('Survival count by Passenger Class')
plt.savefig('eda_pclass.png')
plt.close()

# Age distribution for survived vs. not survived passengers
plt.figure(figsize=(6, 4))
sns.histplot(data=train_df, x='Age', hue='Survived', kde=True)
plt.title('Age distribution by survival')
plt.savefig('eda_age.png')
plt.close()

# Correlation heatmap between the numeric features
plt.figure(figsize=(6, 4))
cor = train_df.drop('Survived', axis=1).corr()
sns.heatmap(cor, annot=True, cmap='coolwarm')
plt.title('Feature correlation')
plt.savefig('eda_corr.png')
plt.close()

## Part 3 - Experiments
You could add as many code cells as needed

In [12]:
X_train = train_df.drop('Survived', axis=1)
y_train = train_df['Survived']

param_grid = [
    {
        'clf': [LogisticRegression(max_iter=1000)],
        'clf__C': [0.1, 1.0, 10.0]
    },
    {
        'clf': [RandomForestClassifier(random_state=42)],
        'clf__n_estimators': [50, 100],
        'clf__max_depth': [5, 10, None]
    }
]

scorer = make_scorer(f1_score)

# Pipeline contains a standard scaler followed by the classifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())  # replaced by grid search
])

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scorer,
    n_jobs=-1,
    refit=True
)

print('\nRunning grid search (this may take a moment)...')

grid.fit(X_train, y_train)

results = pd.DataFrame(grid.cv_results_)
print('\nGrid search results (mean test score):')
print(results[['params', 'mean_test_score', 'std_test_score']])

print('\nBest parameters:', grid.best_params_)
best_model = grid.best_estimator_


Running grid search (this may take a moment)...

Grid search results (mean test score):
                                              params  mean_test_score  \
0  {'clf': LogisticRegression(max_iter=1000), 'cl...         0.733732   
1  {'clf': LogisticRegression(max_iter=1000), 'cl...         0.737571   
2  {'clf': LogisticRegression(max_iter=1000), 'cl...         0.737066   
3  {'clf': RandomForestClassifier(random_state=42...         0.749234   
4  {'clf': RandomForestClassifier(random_state=42...         0.753404   
5  {'clf': RandomForestClassifier(random_state=42...         0.752642   
6  {'clf': RandomForestClassifier(random_state=42...         0.756783   
7  {'clf': RandomForestClassifier(random_state=42...         0.739614   
8  {'clf': RandomForestClassifier(random_state=42...         0.745186   

   std_test_score  
0        0.026388  
1        0.027481  
2        0.023262  
3        0.030696  
4        0.031170  
5        0.021040  
6        0.027894  
7        0.022567  


## Part 4 - Training 
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 3)

In [13]:
best_model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', RandomForestClassifier(max_depth=10, random_state=42))])

## Part 5 - Apply on test and show model performance estimation

In [14]:
X_test = test_df.drop('Survived', axis=1)
y_test = test_df['Survived']
y_pred = best_model.predict(X_test)

test_f1 = f1_score(y_test, y_pred)
print('\nTest F1 score:', test_f1)

print('\nFirst 5 predictions:')
print(pd.DataFrame({'Actual': y_test[:5].values,
                    'Predicted': y_pred[:5]}))


Test F1 score: 0.8275862068965517

First 5 predictions:
   Actual  Predicted
0       0          0
1       0          0
2       0          0
3       0          1
4       1          1
